In [27]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import seaborn as sns
from PIL import Image
import glob
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES']='2'
%matplotlib inline

In [28]:
dataDir = '/data/jupyter/libin713/sample_IDCard'
print(len(os.listdir(dataDir + "/original/")))  #380
print(len(os.listdir(dataDir + "/black/")))     #325
print(len(os.listdir(dataDir + "/coloful/")))   #630

380
325
630


In [29]:
ratio = int(0.7*len(os.listdir('/data/jupyter/libin713/sample_IDCard/original')))
os.listdir('/data/jupyter/libin713/sample_IDCard/original')[266:]

['IMG_20190521_112747.jpg',
 'IMG_0582.JPG',
 'IMG_20190521_112849_2.jpg',
 'IMG_0638.JPG',
 'IMG_0515.JPG',
 'IMG_0538.JPG',
 'IMG_0549.JPG',
 'IMG_0526.JPG',
 'IMG_0618.JPG',
 'IMG_20190521_112750_3.jpg',
 'IMG_20190521_112759_1.jpg',
 'IMG_20190521_112713.jpg',
 'IMG_20190521_112806_1.jpg',
 'IMG_20190521_112849_1.jpg',
 'IMG_20190521_112657_1.jpg',
 'IMG_20190521_112740.jpg',
 'IMG_20190521_112759_2.jpg',
 'IMG_0623.JPG',
 'IMG_20190521_112751.jpg',
 'IMG_0530.JPG',
 'IMG_20190521_112759.jpg',
 'IMG_20190521_112837.jpg',
 'IMG_20190521_112727.jpg',
 'IMG_20190521_112858.jpg',
 'IMG_20190521_112918.jpg',
 'IMG_0617.JPG',
 'IMG_0654.JPG',
 'IMG_0508.JPG',
 'IMG_0544.JPG',
 'IMG_0558.JPG',
 'IMG_0588.JPG',
 'IMG_0629.JPG',
 'IMG_0504.JPG',
 'IMG_0503.JPG',
 'IMG_0613.JPG',
 'IMG_0559.JPG',
 'IMG_20190521_112841_1.jpg',
 'IMG_20190521_112927_1.jpg',
 'IMG_0556.JPG',
 'IMG_20190521_112803.jpg',
 'IMG_20190521_112853.jpg',
 'IMG_20190521_112708_1.jpg',
 'IMG_20190521_112807.jpg',
 'IMG_0

In [33]:
#制作训练集  1134张图片
classes = {'original','black','coloful'}  #图像的类别
writer = tf.python_io.TFRecordWriter("Idcard_train1.tfrecords")

for index, name in enumerate(classes):
    class_path = dataDir + "/" + name + "/"
    ratio = int(len(os.listdir(class_path))*0.85)
    print(ratio)
    for img_name in os.listdir(class_path)[:ratio]:
        img_path = class_path + img_name
        if not os.path.isfile(img_path):
            continue
        img = Image.open(img_path)
        img = img.resize((600,600))
        img_raw = img.tobytes()
        example = tf.train.Example(features = tf.train.Features(feature = {
            "label":tf.train.Feature(int64_list = tf.train.Int64List(value = [index])),
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value = [img_raw]))
        }))
        writer.write(example.SerializeToString())  #序列化为字符串
writer.close()



535
276
323


In [35]:
535+276+323

1134

In [34]:
##制作测试集  201张
classes = {'original','black','coloful'}  #图像的类别
writer = tf.python_io.TFRecordWriter("Idcard_test1.tfrecords")

for index, name in enumerate(classes):
    class_path = dataDir + "/" + name + "/"
    ratio = int(len(os.listdir(class_path))*0.85)
    print(len(os.listdir(class_path))-ratio)
    for img_name in os.listdir(class_path)[ratio:]:
        img_path = class_path + img_name
        if not os.path.isfile(img_path):
            continue
        img = Image.open(img_path)
        img = img.resize((600,600))
        img_raw = img.tobytes()
        example = tf.train.Example(features = tf.train.Features(feature = {
            "label":tf.train.Feature(int64_list = tf.train.Int64List(value = [index])),
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value = [img_raw]))
        }))
        writer.write(example.SerializeToString())  #序列化为字符串
writer.close()
            

95
49
57


In [36]:
95+49+57

201

In [37]:
#观察图片数据[1 0 0]彩色图 [0 1 0] 黑白图  [0 0 1]原件
filename_queue = tf.train.string_input_producer(["Idcard_train1.tfrecords"])  # 读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)  # 返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw': tf.FixedLenFeature([], tf.string),
                                   })  # 取出包含image和label的feature对象
image = tf.decode_raw(features['img_raw'], tf.uint8)
image = tf.reshape(image, [600, 600, 3])
label = tf.cast(features['label'], tf.int32)
label = tf.one_hot(label, 3, 1, 0)
with tf.Session() as sess:  # 开始一个会话
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(800):
        example, l = sess.run([image, label])  # 在会话中取出image和label
        print("%%%%%:",l)
        print("\n")
        img = Image.fromarray(example, 'RGB')  # 这里Image是之前提到的
        img.save(dataDir +'/test1/'+ str(i) + '_''Label_' + str(l) + '.jpg')  # 存下图片
        print(example, l)
    coord.request_stop()
    coord.join(threads)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
%%%%%: [1 0 0]


[[[50 49 44]
  [42 43 37]
  [44 43 38]
  ...
  [27 29 28]
  [21 21 19]
  [29 27 28]]

 [[51 50 46]
  [50 51 45]
  [51 50 46]
  ...
  [25 27 24]
  [32 32 30]
  [33 32 30]]

 [[48 49 44]
  [41 40 36]
  [41 42 37]
  ...
  [35 35 33]
  [21 23 22]
  [34 35 30]]

 ...

 [[50 45 39]
  [53 46 40]
  [53 46 40]
  ...
  [27 22 19]
  [28 23 19]
  [32 27 23]]

 [[55 51 42]
  [55 50 44]
  [55 48 42]
  ...
  [30 25 19]
  [23 20 13]
  [28 21 15]]

 [[54 47 41]
  [48 41 35]
  [51 46 40]
  ...
  [29 21 18]
  [29 20 15]
  [27 24 19]]] [1 0 0]
%%%%%: [1 0 0]


[[[37 34 29]
  [36 35 31]
  [36 33 28]
  ...
  [33 34 29]
  [26 30 29]
  [29 31 28]]

 [[32 31 26]
  [41 40 38]
  [41 37 34]
  ...
  [21 22 17]
  [26 31 27]
  [21 23 20]]

 [[51 48 39]
  [89 88 83]
  [33 29 28]
  ...
  [22 23 18]
  [27 29 26]
  [30 30 28]]

 ...

 [[45 41 38]
  [41 37 36]
  [34 34 32]
  ...
  [ 8  8  6]
  [ 8  8  6]
  [11 13 10]]

 [[42 34 32]

[[[ 71  82  78]
  [ 66  77  71]
  [ 75  75  73]
  ...
  [ 15  16  11]
  [ 14  16  13]
  [ 13  15  12]]

 [[ 72  78  74]
  [ 73  78  74]
  [ 70  79  74]
  ...
  [ 14  15  10]
  [ 17  19  16]
  [ 15  17  14]]

 [[ 71  73  72]
  [ 72  78  76]
  [ 71  77  73]
  ...
  [ 12  14   9]
  [ 12  14  11]
  [ 12  14   9]]

 ...

 [[119  73  73]
  [124  81  65]
  [121  76  57]
  ...
  [ 19  19  17]
  [ 25  25  23]
  [ 19  19  17]]

 [[127  77  66]
  [125  78  62]
  [125  78  60]
  ...
  [ 16  16  14]
  [ 14  16  13]
  [ 17  19  16]]

 [[133  84  77]
  [126  81  62]
  [121  79  57]
  ...
  [ 18  20  17]
  [ 14  16  13]
  [ 15  17  14]]] [1 0 0]
%%%%%: [1 0 0]


[[[ 44  40  28]
  [ 37  37  27]
  [ 39  36  27]
  ...
  [ 52  46  34]
  [ 58  52  38]
  [ 60  53  43]]

 [[ 39  40  26]
  [ 36  36  26]
  [ 40  40  30]
  ...
  [ 54  48  36]
  [ 54  48  34]
  [ 57  51  39]]

 [[ 39  36  27]
  [ 35  35  25]
  [ 34  34  24]
  ...
  [ 57  51  39]
  [ 60  57  42]
  [ 55  52  37]]

 ...

 [[177 111  77]
  [182 115 

[[[53 53 45]
  [35 36 31]
  [44 43 39]
  ...
  [23 23 21]
  [22 22 20]
  [30 27 22]]

 [[44 43 39]
  [44 44 42]
  [39 38 36]
  ...
  [23 22 20]
  [14 14 14]
  [30 26 23]]

 [[46 45 43]
  [45 44 42]
  [45 44 42]
  ...
  [23 23 21]
  [17 17 17]
  [40 39 35]]

 ...

 [[54 47 41]
  [44 37 31]
  [48 44 35]
  ...
  [26 21 18]
  [25 17 14]
  [23 18 14]]

 [[55 48 40]
  [47 43 34]
  [46 42 33]
  ...
  [24 19 15]
  [27 24 17]
  [40 31 26]]

 [[51 46 40]
  [51 46 40]
  [46 39 33]
  ...
  [28 25 18]
  [26 23 16]
  [28 21 15]]] [1 0 0]
%%%%%: [1 0 0]


[[[221 206 173]
  [221 207 172]
  [219 206 171]
  ...
  [229 217 179]
  [229 216 181]
  [229 218 188]]

 [[220 207 173]
  [217 204 169]
  [221 207 172]
  ...
  [229 217 179]
  [233 220 185]
  [233 221 183]]

 [[220 207 173]
  [221 208 173]
  [221 207 172]
  ...
  [232 219 184]
  [229 216 181]
  [228 215 180]]

 ...

 [[211 196 157]
  [210 195 156]
  [205 190 151]
  ...
  [173 155 117]
  [176 158 118]
  [175 157 117]]

 [[212 195 152]
  [211 193 155]

[[[225  70  52]
  [222  69  53]
  [229  73  60]
  ...
  [198  51  33]
  [196  51  32]
  [197  52  33]]

 [[227  75  54]
  [227  75  54]
  [225  72  54]
  ...
  [201  54  36]
  [201  54  36]
  [204  57  37]]

 [[228  76  62]
  [228  77  56]
  [232  81  60]
  ...
  [198  51  35]
  [200  53  37]
  [202  55  39]]

 ...

 [[232 164 155]
  [231 163 152]
  [223 155 142]
  ...
  [184  47  28]
  [189  44  27]
  [187  40  22]]

 [[232 164 151]
  [230 163 147]
  [228 160 147]
  ...
  [191  52  31]
  [188  46  24]
  [187  40  22]]

 [[228 160 149]
  [223 153 141]
  [229 160 145]
  ...
  [184  43  25]
  [190  42  32]
  [190  42  28]]] [1 0 0]
%%%%%: [1 0 0]


[[[226  68  56]
  [225  75  61]
  [221  69  56]
  ...
  [201  48  34]
  [201  48  32]
  [203  53  29]]

 [[231  81  64]
  [228  77  56]
  [224  73  56]
  ...
  [197  46  29]
  [198  45  29]
  [194  41  25]]

 [[231  82  62]
  [226  76  59]
  [227  76  57]
  ...
  [204  53  34]
  [203  50  32]
  [200  47  31]]

 ...

 [[218 130 120]
  [218 131 

[[[78 78 70]
  [78 76 64]
  [73 73 63]
  ...
  [53 54 49]
  [56 57 49]
  [53 54 46]]

 [[76 73 64]
  [77 77 67]
  [79 76 67]
  ...
  [56 57 51]
  [60 61 53]
  [55 56 48]]

 [[71 71 61]
  [76 76 66]
  [73 69 60]
  ...
  [59 60 52]
  [52 53 45]
  [59 60 52]]

 ...

 [[77 76 71]
  [75 76 71]
  [79 78 73]
  ...
  [17 19 18]
  [20 22 21]
  [16 16 16]]

 [[73 72 68]
  [66 67 62]
  [74 74 72]
  ...
  [20 22 21]
  [14 16 15]
  [16 16 16]]

 [[78 77 73]
  [80 80 78]
  [78 79 73]
  ...
  [19 21 20]
  [12 14 13]
  [18 18 18]]] [1 0 0]
%%%%%: [1 0 0]


[[[ 27  28  22]
  [ 26  28  17]
  [ 25  27  16]
  ...
  [ 49  43  31]
  [ 52  44  31]
  [ 46  40  26]]

 [[ 28  30  19]
  [ 25  27  14]
  [ 28  30  19]
  ...
  [ 51  45  33]
  [ 48  42  28]
  [ 45  41  29]]

 [[ 28  28  18]
  [ 28  30  19]
  [ 25  27  16]
  ...
  [ 49  41  30]
  [ 47  44  29]
  [ 47  48  34]]

 ...

 [[154  79  40]
  [153  84  43]
  [162  93  52]
  ...
  [  8   9   4]
  [ 10  11   6]
  [ 10   9   5]]

 [[157  83  48]
  [157  89  44]

...

[[[146 158 170]
  [164 177 186]
  [160 174 183]
  ...
  [172 184 196]
  [174 186 200]
  [173 185 199]]

 [[171 181 191]
  [158 171 180]
  [164 177 186]
  ...
  [172 184 196]
  [175 187 201]
  [172 184 198]]

 [[171 184 193]
  [162 175 184]
  [162 175 184]
  ...
  [172 184 196]
  [175 184 199]
  [177 186 201]]

 ...

 [[139 143 152]
  [131 138 146]
  [140 147 155]
  ...
  [181 191 201]
  [183 196 205]
  [185 195 204]]

 [[152 159 165]
  [138 145 151]
  [142 151 158]
  ...
  [180 190 200]
  [183 196 205]
  [179 192 200]]

 [[140 151 153]
  [147 152 158]
  [141 152 158]
  ...
  [187 197 207]
  [179 193 202]
  [179 194 201]]] [0 1 0]
%%%%%: [0 1 0]


[[[ 47  36  18]
  [ 50  37  20]
  [ 55  39  23]
  ...
  [203 205 181]
  [207 209 188]
  [206 208 187]]

 [[ 47  36  18]
  [ 49  36  19]
  [ 54  38  22]
  ...
  [198 200 176]
  [204 206 185]
  [207 209 188]]

 [[ 47  36  18]
  [ 49  36  19]
  [ 54  38  22]
  ...
  [206 208 184]
  [204 206 185]
  [205 207 186]]

 ...

 [[187 167 134]
  [187 167 

[[[107  17   6]
  [110  18   7]
  [105  13   2]
  ...
  [160  27   8]
  [164  29  10]
  [164  29  10]]

 [[109  17   6]
  [108  16   5]
  [108  16   3]
  ...
  [164  33  13]
  [163  32  12]
  [163  32  12]]

 [[109  17   6]
  [108  16   5]
  [106  17   3]
  ...
  [161  31   9]
  [163  33  11]
  [161  31   9]]

 ...

 [[174  66  37]
  [173  68  36]
  [181  73  44]
  ...
  [190  64  39]
  [192  68  42]
  [191  67  41]]

 [[175  67  38]
  [171  66  34]
  [179  74  42]
  ...
  [189  65  39]
  [199  75  49]
  [191  65  40]]

 [[175  68  36]
  [172  67  35]
  [174  71  36]
  ...
  [191  67  41]
  [192  67  39]
  [193  68  40]]] [0 1 0]
%%%%%: [0 1 0]


[[[ 98 106 109]
  [ 95 105 106]
  [108 116 119]
  ...
  [ 29  20  11]
  [ 28  20   7]
  [ 35  27  14]]

 [[ 94 104 106]
  [ 93 103 104]
  [ 97 102 106]
  ...
  [ 29  23  11]
  [ 36  30  16]
  [ 29  23   9]]

 [[ 92 104 104]
  [ 92 104 104]
  [ 91  99 102]
  ...
  [ 33  27  15]
  [ 31  25  11]
  [ 31  25  11]]

 ...

 [[162 180 192]
  [163 183 

[[[ 91  91  79]
  [103 104  90]
  [107 104  89]
  ...
  [149 149 139]
  [147 149 138]
  [147 150 139]]

 [[104 102  89]
  [104 102  89]
  [111 109  96]
  ...
  [148 150 137]
  [151 153 140]
  [148 150 139]]

 [[107 103  91]
  [106 104  91]
  [108 107  89]
  ...
  [152 154 141]
  [148 152 138]
  [147 150 139]]

 ...

 [[156 157 151]
  [154 157 148]
  [158 159 151]
  ...
  [172 172 162]
  [174 174 166]
  [170 170 160]]

 [[154 157 150]
  [156 157 149]
  [154 157 146]
  ...
  [171 171 163]
  [171 172 164]
  [170 170 162]]

 [[160 162 157]
  [154 155 147]
  [154 157 148]
  ...
  [171 172 164]
  [169 170 162]
  [170 170 162]]] [0 1 0]
%%%%%: [0 1 0]


[[[106  12   4]
  [107  19   7]
  [104  11   3]
  ...
  [159  26   7]
  [158  22   6]
  [159  27   6]]

 [[110  18   7]
  [103  15   3]
  [111  21  12]
  ...
  [160  27   8]
  [162  26  10]
  [158  25   6]]

 [[107  15   4]
  [106  16   5]
  [106  14   3]
  ...
  [161  28   9]
  [159  26   9]
  [161  25   9]]

 ...

 [[174  67  35]
  [173  66 

[[[196 192 163]
  [197 192 163]
  [198 193 164]
  ...
  [197 195 170]
  [199 197 174]
  [202 200 177]]

 [[196 192 165]
  [197 191 165]
  [198 192 166]
  ...
  [198 196 171]
  [201 199 176]
  [200 198 175]]

 [[197 193 168]
  [199 193 169]
  [198 192 168]
  ...
  [197 195 170]
  [199 197 174]
  [201 201 177]]

 ...

 [[192 178 143]
  [191 177 142]
  [193 179 144]
  ...
  [  9   9   7]
  [  9   9   7]
  [  8   8   6]]

 [[193 179 144]
  [193 179 144]
  [193 179 144]
  ...
  [ 11  11   9]
  [  9   9   7]
  [  8   8   6]]

 [[190 177 145]
  [189 176 144]
  [190 176 141]
  ...
  [ 11  11   9]
  [  9   9   7]
  [  8   8   6]]] [0 1 0]
%%%%%: [0 1 0]


[[[ 35  26  11]
  [ 50  37  18]
  [ 53  38  17]
  ...
  [162  53  33]
  [163  56  38]
  [172  62  45]]

 [[ 39  30  15]
  [ 49  36  17]
  [ 54  39  18]
  ...
  [162  53  33]
  [161  54  36]
  [164  54  37]]

 [[ 42  33  18]
  [ 52  39  20]
  [ 55  40  19]
  ...
  [166  57  37]
  [162  55  37]
  [166  56  39]]

 ...

 [[ 29  16   0]
  [ 31  18 

[[[177 184 202]
  [173 185 201]
  [173 185 199]
  ...
  [177 192 211]
  [172 184 200]
  [175 186 204]]

 [[178 185 203]
  [175 187 203]
  [172 184 198]
  ...
  [174 189 208]
  [175 187 203]
  [175 186 204]]

 [[176 183 201]
  [176 188 204]
  [174 186 200]
  ...
  [172 187 206]
  [175 187 203]
  [175 186 204]]

 ...

 [[148 155 171]
  [154 161 177]
  [148 155 171]
  ...
  [ 84  90 102]
  [ 85  91 103]
  [ 80  86  98]]

 [[146 153 169]
  [145 152 168]
  [144 151 167]
  ...
  [ 81  87  99]
  [ 83  89 101]
  [ 88  95 105]]

 [[142 148 162]
  [147 153 167]
  [145 151 167]
  ...
  [ 86  90 102]
  [ 81  87  99]
  [ 77  86  93]]] [0 1 0]
%%%%%: [0 1 0]


[[[116 133 141]
  [119 130 134]
  [116 122 122]
  ...
  [ 31  24   8]
  [ 32  23   8]
  [ 32  23   8]]

 [[119 129 138]
  [112 122 132]
  [ 64  72  75]
  ...
  [ 31  24   8]
  [ 34  27  11]
  [ 35  26   9]]

 [[112 127 134]
  [113 124 128]
  [ 42  50  53]
  ...
  [ 30  23   7]
  [ 32  25   9]
  [ 32  25   9]]

 ...

 [[167 144 150]
  [123 144 

In [26]:
print(label)

Tensor("Cast_1:0", shape=(), dtype=int32)


In [4]:

N_CLASSES = 2
IMG_W = 208  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 208
BATCH_SIZE = 16
CAPACITY = 2000
MAX_STEP = 10000 # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001 # with current parameters, it is suggested to use learning rate<0.0001

logs_train_dir = dataDir + '/logs/recordstrain/'
#    
#    train, train_label = input_data.get_files(train_dir)
tfrecords_file = 'Idcard_train.tfrecords'
train_batch, train_label_batch = read_and_decode(tfrecords_file, batch_size=BATCH_SIZE)
train_batch = tf.cast(train_batch,dtype=tf.float32)
train_label_batch = tf.cast(train_label_batch,dtype=tf.int64)

In [ ]:
N_CLASSES = 2
IMG_W = 208  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 208
BATCH_SIZE = 16
CAPACITY = 2000
MAX_STEP = 10000 # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001 # with current parameters, it is suggested to use learning rate<0.0001

import create_records as cr

def run_training1():
    
    # you need to change the directories to yours.
#    train_dir = '/home/hjxu/PycharmProjects/01_cats_vs_dogs/data/train/'
    logs_train_dir = dataDir + '/logs/recordstrain/'
#    
#    train, train_label = input_data.get_files(train_dir)
    tfrecords_file = 'Idcard_train.tfrecords'
    train_batch, train_label_batch = cr.read_and_decode(tfrecords_file, batch_size=BATCH_SIZE)
    train_batch = tf.cast(train_batch,dtype=tf.float32)
    train_label_batch = tf.cast(train_label_batch,dtype=tf.int64)
    
    train_logits = model.inference(train_batch, BATCH_SIZE, N_CLASSES)
    train_loss = model.losses(train_logits, train_label_batch)        
    train_op = model.trainning(train_loss, learning_rate)
    train__acc = model.evaluation(train_logits, train_label_batch)
       
    summary_op = tf.summary.merge_all()
    sess = tf.Session()
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                    break
            _, tra_loss, tra_acc = sess.run([train_op, train_loss, train__acc])
               
            if step % 50 == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)
            
            if step % 2000 == 0 or (step + 1) == MAX_STEP:
                checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
                
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)
    sess.close()
    
run_training1()
